# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
df = pd.read_csv("cities.csv")
df = df.drop(columns=["Unnamed: 0"])
df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,bajos de haina,18.4154,-70.0332,25.56,94,20,2.06,DO,1616471944
1,busselton,-33.6500,115.3333,16.11,71,45,0.89,AU,1616471945
2,east london,-33.0153,27.9116,16.00,94,0,2.06,ZA,1616471945
3,groningen,53.2500,6.7500,4.44,93,90,3.09,NL,1616471945
4,boyolangu,-8.1181,111.8935,29.86,64,95,2.72,ID,1616471945
...,...,...,...,...,...,...,...,...,...
547,tessaoua,13.7574,7.9874,26.60,16,31,4.00,NE,1616472087
548,ciudad bolivar,8.1222,-63.5497,27.02,65,72,5.31,VE,1616472087
549,erdenet,49.0333,104.0833,3.82,41,87,1.80,MN,1616472088
550,revelganj,25.7833,84.6667,28.00,34,20,2.06,IN,1616472088


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
gmaps.configure(api_key=gkey)

locations = df[['Lat', 'Lng']]
weight = df['Humidity']

In [12]:
figure = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(locations, weights = weight)
figure.add_layer(heatmap_layer)
figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [19]:
hotel = df.loc[(df["Max Temp"] < 30) & 
                        (df["Max Temp"] > 20) & 
                        (df['Wind Speed'] < 10) &
                       (df['Cloudiness'] < 8)]

hotel_df = hotel.dropna()


In [20]:
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
16,mecca,21.4267,39.8261,26.72,40,0,1.19,SA,1616471948
29,lazaro cardenas,17.9583,-102.2000,22.35,66,0,1.92,MX,1616471951
38,abnub,27.2696,31.1511,23.00,27,0,4.12,EG,1616471953
41,rikitea,-23.1203,-134.9692,26.86,73,7,5.53,PF,1616471954
45,atuona,-9.8000,-139.0333,27.03,77,7,9.05,PF,1616471955


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:
hotel_df["Coordinates"] = hotel_df["Lat"].astype(str) + ", " + hotel_df["Lng"].astype(str)
hotel_df["Hotel Name"] = ""

hotel_df = hotel_df.reset_index()
hotel_df.head()

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Coordinates,Hotel Name
0,16,mecca,21.4267,39.8261,26.72,40,0,1.19,SA,1616471948,"21.4267, 39.8261",
1,29,lazaro cardenas,17.9583,-102.2000,22.35,66,0,1.92,MX,1616471951,"17.9583, -102.2",
2,38,abnub,27.2696,31.1511,23.00,27,0,4.12,EG,1616471953,"27.2696, 31.1511",
3,41,rikitea,-23.1203,-134.9692,26.86,73,7,5.53,PF,1616471954,"-23.1203, -134.9692",
4,45,atuona,-9.8000,-139.0333,27.03,77,7,9.05,PF,1616471955,"-9.8, -139.0333",


In [25]:
hotel_lat = hotel_df["Lat"].tolist()
hotel_long = hotel_df["Lng"].tolist()
coord = hotel_df['Coordinates'].tolist()
hotel_list = []

Lat_Long = list(zip(hotel_lat, hotel_long))
target = "Hotel"

base = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
radius = 5000

for coords in coord: 
    try:
        parameters = {"location": coords,
          "keyword": target, 
          "key": gkey,
          "radius": radius}

        response_hotel = requests.get(base, parameters).json()
    
        #hotell = response_hotel['results'][0]['name']
        print(response_hotel['results'][0]['name'])
        hotel_list.append(response_hotel['results'][0]['name'])
        #pprint(response_hotel)
    except IndexError:
        hotel_list.append("No hotels available")
        print("No hotels available")

Copthorne Makkah Al Naseem Hotel
City Express Lázaro Cárdenas
No hotels available
Pension Bianca & Benoit
Hôtel Hiva Oa Hanakee Lodge
Marina Hotel
Sur Plaza Hotel
Florence Hotel Assiut
Lodge Outback Motel
Golden Tulip Porto Vitória
Star Emirates garden villas
Birdland Beach Club and BPO Digital Online Live Work Beach Resort
Les Embruns Du Baril
Centre Amadou Seck - Linguere
The Westin Resort & Spa, Puerto Vallarta
No hotels available
Hospitality Carnarvon
Ocean Centre Hotel
Centro Olaya
Hotel Pissoloto
Hotel Del Carmen
No hotels available
ป่ายางรีสอร์ท
Sofitel Legend Old Cataract Aswan
Pousada Kactus
Nizwa Residence Hotel Apartments
No hotels available
Sunyata Hotel
Auberge mer et desert
No hotels available
HOSTAL MIRAMAR
Desert Gat
Grand Velas Riviera Nayarit
No hotels available
Morritt Farms
Lakefront Lodge Te Anau
No hotels available
Minab Tourism Hotel
No hotels available


In [26]:
hotel_df["Hotel Name"] = hotel_list
hotel_df.head()

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Coordinates,Hotel Name
0,16,mecca,21.4267,39.8261,26.72,40,0,1.19,SA,1616471948,"21.4267, 39.8261",Copthorne Makkah Al Naseem Hotel
1,29,lazaro cardenas,17.9583,-102.2000,22.35,66,0,1.92,MX,1616471951,"17.9583, -102.2",City Express Lázaro Cárdenas
2,38,abnub,27.2696,31.1511,23.00,27,0,4.12,EG,1616471953,"27.2696, 31.1511",No hotels available
3,41,rikitea,-23.1203,-134.9692,26.86,73,7,5.53,PF,1616471954,"-23.1203, -134.9692",Pension Bianca & Benoit
4,45,atuona,-9.8000,-139.0333,27.03,77,7,9.05,PF,1616471955,"-9.8, -139.0333",Hôtel Hiva Oa Hanakee Lodge


In [27]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [28]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
figure.add_layer(markers)

# Display figure
figure

Figure(layout=FigureLayout(height='420px'))